In [1]:
import tensorflow as tf
import pickle
import os
import shutil
import tempfile
import datetime
from glob import iglob
import time
# import attention
from collections import deque
import pickle
import pyreader
import numpy as np
from attention import Batcher, construct_feed_dict, extract_results, get_evals,AttentionModel,get_initial_state

In [30]:
type_max_rands = {
    "var": 4750, "function": 2900, "Class": 440, "attribute": 2400, "arg": 2400
}
def map_token(map, token):
    mask = 0
    if token.startswith("(*) "):
        mask = 1
        token = token.replace("(*) ", "")

    if token in map:
        return map[token], mask

    # Not in map, is it an identifier?
    if "|" in token:
        spl = token.split("|")
        if spl[1] in map:
            return map[spl[1]]
        elif spl[0] in map:
            return map[spl[0]]

    elif any([token.startswith(prefix) for prefix in [key for key in type_max_rands]]):
        return pyreader.oov_id

    raise KeyError(token)

In [52]:
seq_length = 1
batch_size = 1
hidden_size = 50
num_samples = 3
lambda_type='state'
max_attention=3
model_path = './model/2017-07-01--01-22--006942'
data_path='data_samples/mapping.map'
with open(data_path, "rb") as f:
    word_to_id = pickle.load(f)
vocab_size = len(word_to_id)
inv_map = {v: k for k, v in word_to_id.items()}

In [89]:
all_test_cases = ["import", "os", "\n", "class", "(*) Class291", ":", "\n", "§<indent>§", "def", "__init__", "(", "self", ",",
     "(*) arg123", ")", ":", "\n", "§<indent>§", "self", ".", "(*) attribute|attribute172", "=", "arg123", "\n", "\n",
     "§<dedent>§",
     "def", "(*) function234", "(", "self", ",", "(*) filename|arg432", ")", ":", "\n", "§<indent>§", "with", "open",
     "(", "filename|arg432", ",", "'r'", ")", "as", "(*) f|var76", ":", "\n", "§<indent>§", "(*) lines|var91", "=",
     "f|var76", ".",
     "readlines", "(", ")", "\n", "§<dedent>§", "return", "len", "(", "lines|var91", ")", "\n", "\n", "§<dedent>§",
     "def", "(*) func|function921", "(", "self", ",", "(*) arg191", ")", ":", "\n", "§<indent>§", "(*) var543", "=",
     "os", ".", "path", ".", "join", "(", "self", "."]

In [93]:
with tf.Graph().as_default(), tf.Session() as session:
    masks_ = tf.placeholder(tf.bool, [seq_length, batch_size, 1], name="masks")
    input_data_ = tf.placeholder(tf.int32, [seq_length, batch_size], name="inputs")
    targets_ = tf.placeholder(tf.float32, [seq_length, batch_size], name="targets")
    
    a = AttentionModel(input_data=input_data_,
                                 targets=targets_,
                                 masks=masks_,
                                 is_training=False,
                                 attention_num= 1,
                                 batch_size=batch_size,
                                 hidden_size=hidden_size,
                                 num_samples=num_samples,
                                 seq_length=seq_length,
                                 vocab_size=vocab_size,
                                 lambda_type=lambda_type,
                                 max_attention=max_attention)
    
    
    variables = tf.trainable_variables()
    session.run(tf.global_variables_initializer())
    saver = tf.train.Saver(variables)
    saver.restore(session, os.path.join(model_path, "model.tf"))
    
    prediction_op = tf.nn.top_k(a.predict, 5)
    
    to_eval = [prediction_op[0], prediction_op[1]]
    evaluation = get_evals(to_eval, a)
    
    state, att_states, att_ids, att_counts = get_initial_state(a, session)
    
    for i, token in enumerate(all_test_cases[:-5]):
        lst =np.ones([1, 1])
        token_id = map_token(word_to_id, token)
        
        if isinstance(token_id, tuple):
            data = (np.array([[token_id[0]]]), np.array([[1]]), np.array([lst]), np.array([1]), np.array([1]))
        else:
            data = (np.array([[token_id]]), np.array([[1]]), np.array([lst]), np.array([1]), np.array([1]))
        feed_dict, _ = construct_feed_dict(a, data, state, att_states, att_ids, att_counts)
        
        results = session.run(evaluation, feed_dict)
#         results_match = ''.join(str(results[1][0][0]))
#         actual = [map_token(word_to_id, t) for t in all_test_cases[i + 1:i + 5 + 1]]
#         act = " ".join([inv_map[t[0]].replace("\n", "<newline>") for t in actual])
#         print('=============')
        print('predict', inv_map[results[1][0][1]])
#         print('real', act)

Constructing Attention Cell
INFO:tensorflow:Restoring parameters from ./model/2017-07-01--01-22--006942\model.tf
predict import
predict os
predict 

predict class
predict Class291
predict :
predict 

predict §<indent>§
predict def
predict __init__
predict §PAD§
predict self
predict §PAD§
predict §OOV§
predict )
predict :
predict 

predict §<indent>§
predict self
predict §PAD§


KeyError: 'attribute|attribute172'